In [1]:
# try to create and export a simple fbx scene...

# this example should cover:
# - geometry (vertices, faces, normals)
# - uvs
# - textures
# - materials
# - a simple animation

import fbx

In [2]:
def print_normals_info(mesh: fbx.FbxMesh):
    print('printing normals info')
    normal_element: fbx.FbxLayerElementNormal = mesh.GetElementNormal()
    
    if normal_element is not None:
        print('found normal element of the mesh')
    else:
        print('no normal element of the mesh found')
        return
    
    reference_mode = normal_element.GetReferenceMode()
    
    if reference_mode == fbx.FbxLayerElementNormal.eDirect:
        print('normal element has direct reference mode')
    elif reference_mode == fbx.FbxLayerElementNormal.eIndexToDirect:
        print('normal element has index to direct reference mode')
    else:
        print('normal element has unhandled reference mode' + str(reference_mode))
        return
    
    if normal_element.GetMappingMode() == fbx.FbxLayerElementNormal.eByControlPoint:
        print('normal element mapping is by control point, printing normals')
        
        for control_point_index in range(0, mesh.GetControlPointsCount()):
            
            if reference_mode == fbx.FbxLayerElementNormal.eDirect:
                normal_index = control_point_index
            elif reference_mode == fbx.FbxLayerElementNormal.eIndexToDirect:
                normal_index = normal_element.GetIndexArray().GetAt(control_point_index)
                
            normal = normal_element.GetDirectArray().GetAt(normal_index)
            print(str(normal))
    else:
        print('unhandled element mapping ' + str(normal_element.GetMappingMode()))
        return

In [24]:
memory_manager = fbx.FbxManager.Create()
scene = fbx.FbxScene.Create(memory_manager, '')

mesh_node = fbx.FbxNode.Create(memory_manager, 'cube')
scene.GetRootNode().AddChild(mesh_node)

# -- mesh
mesh_attribute: fbx.FbxMesh = fbx.FbxMesh.Create(memory_manager, '')
mesh_node.AddNodeAttribute(mesh_attribute)

# -- mesh -- vertices
mesh_attribute.InitControlPoints(8)
CUBE_VERTS = (
    (0,0,0),
    (0,0,1),
    (0,1,0),
    (0,1,1),
    (1,0,0),
    (1,0,1),
    (1,1,0),
    (1,1,1)
)
for i in range(0, 8):
    vert = CUBE_VERTS[i]
    mesh_attribute.SetControlPointAt(fbx.FbxVector4(vert[0], vert[1], vert[2], 0.0), i)

# -- mesh -- faces
CUBE_FACES = (
    (0,1,3,2),
    (3,2,6,7),
    (4,5,7,6),
    (0,1,5,4),
    (1,3,7,5),
    (0,2,6,4)
)
for i in range(0, len(CUBE_FACES)):
    mesh_attribute.BeginPolygon()
    mesh_attribute.AddPolygon(CUBE_FACES[i][0])
    mesh_attribute.AddPolygon(CUBE_FACES[i][1])
    mesh_attribute.AddPolygon(CUBE_FACES[i][2])
    mesh_attribute.AddPolygon(CUBE_FACES[i][3])
    mesh_attribute.EndPolygon()

# -- mesh -- normals
def normals_per_polygon_vertex(normal_element: fbx.FbxLayerElementNormal):
    
    NORMALS = (
        (-1.0, 0.0, 0.0, 0.0),
        (-1.0, 0.0, 0.0, 0.0),
        (-1.0, 0.0, 0.0, 0.0),
        (-1.0, 0.0, 0.0, 0.0),
        (0.0, 1.0, 0.0, 0.0),
        (0.0, 1.0, 0.0, 0.0),
        (0.0, 1.0, 0.0, 0.0),
        (0.0, 1.0, 0.0, 0.0),
        (1.0, 0.0, 0.0, 0.0),
        (1.0, 0.0, 0.0, 0.0),
        (1.0, 0.0, 0.0, 0.0),
        (1.0, 0.0, 0.0, 0.0),
        (0.0, -1.0, 0.0, 0.0),
        (0.0, -1.0, 0.0, 0.0),
        (0.0, -1.0, 0.0, 0.0),
        (0.0, -1.0, 0.0, 0.0),
        (0.0, 0.0, 1.0, 0.0),
        (0.0, 0.0, 1.0, 0.0),
        (0.0, 0.0, 1.0, 0.0),
        (0.0, 0.0, 1.0, 0.0),
        (0.0, 0.0, -1.0, 0.0),
        (0.0, 0.0, -1.0, 0.0),
        (0.0, 0.0, -1.0, 0.0),
        (0.0, 0.0, -1.0, 0.0)
    )

    normal_element.SetMappingMode(fbx.FbxLayerElementNormal.eByPolygonVertex)
    normal_element.SetReferenceMode(fbx.FbxLayerElementNormal.eDirect)

    for i in range(0, 24):
        normal_element.GetDirectArray().Add(fbx.FbxVector4(NORMALS[i][0], NORMALS[i][1], NORMALS[i][2]))
 
def normals_per_polygon(normal_element: fbx.FbxLayerElementNormal):
    NORMALS = (
        (-1.0, 0.0, 0.0, 0.0),
        (0.0, 1.0, 0.0, 0.0),
        (1.0, 0.0, 0.0, 0.0),
        (0.0, -1.0, 0.0, 0.0),
        (0.0, 0.0, 1.0, 0.0),
        (0.0, 0.0, -1.0, 0.0)
    )
    
    normal_element.SetMappingMode(fbx.FbxLayerElementNormal.eByPolygon)
    normal_element.SetReferenceMode(fbx.FbxLayerElementNormal.eDirect)
    
    for i in range(0, 6):
        normal_element.GetDirectArray().Add(fbx.FbxVector4(NORMALS[i][0], NORMALS[i][1], NORMALS[i][2]))

normal_elem: fbx.FbxLayerElementNormal = mesh_attribute.CreateElementNormal()
normals_per_polygon_vertex(normal_elem)

if mesh_attribute.GetLayer(0) is None:
    mesh_attribute.CreateLayer()
    print('new layer created')
layer: fbx.FbxLayer = mesh_attribute.GetLayer(0)

layer.SetNormals(normal_elem)

# -- mesh -- uvs

uvs = (
    ((0, 1.0/3), (0.25, 1.0/3), (0.25, 2.0/3), (0, 2.0/3)),
    ((0.25, 1.0/3), (0.5, 1.0/3), (0.5, 2.0/3), (0.25, 2.0/3)),
    ((0.5, 1.0/3), (0.75, 1.0/3), (0.75, 2.0/3), (0.5, 2.0/3)),
    ((0.5, 2.0/3), (0.75, 2.0/3), (0.75, 1.0), (0.5, 1.0)),
    ((0.5, 0), (0.75, 0), (0.75, 1.0/3), (0.5, 1.0/3)),
    ((0.75, 1.0/3), (1.0, 1.0/3), (1.0, 2.0/3), (0.75, 2.0/3))
)

uv_element: fbx.FbxLayerElementUV = mesh_attribute.CreateElementUV("MyUVSet1")
uv_element.SetMappingMode(fbx.FbxLayerElement.eByPolygonVertex)
uv_element.SetReferenceMode(fbx.FbxLayerElement.eDirect)

for uv in uvs:
    for uv_coord in uv:
        uv_element.GetDirectArray().Add(fbx.FbxVector2(uv_coord[0], uv_coord[1]))

# -- material
material = fbx.FbxSurfacePhong.Create(memory_manager, "dice_mat")
material.Emissive.Set(fbx.FbxDouble3(0.0, 0.0, 0.0))
material.Ambient.Set(fbx.FbxDouble3(1.0, 0.0, 0.0))
material.Diffuse.Set(fbx.FbxDouble3(0.75, 0.75, 0.0))
material.DiffuseFactor.Set(1.0)
material.ShadingModel.Set("Phong")
material.Shininess.Set(0.5)
mesh_node.AddMaterial(material)

# -- textures -- diffuse
diffuse_texture: fbx.FbxFileTexture = fbx.FbxFileTexture.Create(memory_manager, "diffuse")
diffuse_texture.SetFileName("dice_diffuse.png")
diffuse_texture.SetTextureUse(fbx.FbxTexture.eStandard)
diffuse_texture.SetMappingType(fbx.FbxTexture.eUV)
diffuse_texture.SetMaterialUse(fbx.FbxFileTexture.eModelMaterial)
diffuse_texture.SetSwapUV(False)
diffuse_texture.SetScale(1.0, 1.0)

material.Diffuse.ConnectSrcObject(diffuse_texture)

# EXPORT
exporter = fbx.FbxExporter.Create(memory_manager, '')
exporter.Initialize('cuby', -1, memory_manager.GetIOSettings())
result = exporter.Export(scene)

if result:
    print('exported succesfully')

memory_manager.Destroy()

exported succesfully


In [7]:
# the normals are obviously set correctly, but are they correct when reimporting the exported model?
memory_manager = fbx.FbxManager.Create()
imported_scene: fbx.FbxScene = fbx.FbxScene.Create(memory_manager, '')

fbx_importer: fbx.FbxImporter = fbx.FbxImporter.Create(memory_manager, '')
fbx_importer.Initialize('cuby.fbx', -1, memory_manager.GetIOSettings())
fbx_importer.Import(imported_scene)

print_normals_info(imported_scene.GetRootNode().GetChild(0).GetMesh())

memory_manager.Destroy()

# the normals are exported perfectly!!! why not read properly by blender?? need a proper fbx viewer to be sure...

printing normals info
found normal element of the mesh
normal element has direct reference mode
unhandled element mapping 3


In [13]:
# exploring the blender cube
memory_manager: fbx.FbxManager = fbx.FbxManager.Create()

importer = fbx.FbxImporter.Create(memory_manager, '')
scene: fbx.FbxScene = fbx.FbxScene.Create(memory_manager, '')

importer.Initialize('blen_cube.fbx', -1, memory_manager.GetIOSettings())
importer.Import(scene)

mesh_node: fbx.FbxNode = scene.GetRootNode().GetChild(0)
mesh: fbx.FbxMesh = mesh_node.GetMesh()

if mesh is None:
    print('mesh is None')
    
# control points
print(str(mesh.GetControlPointsCount()) + ' control points in this mesh')

normal_placeholder = fbx.FbxVector4(0.0, 0.0, 0.0, 0.0)
mesh.GetPolygonVertexNormal(0,0,normal_placeholder)
print(normal_placeholder)

# normals
layer: fbx.FbxLayer = mesh.GetLayer(0)
if layer is None:
    print('no layers found in the mesh')
else:
    normals = layer.GetNormals()
    if normals is not None:
        print('mesh layer 0 has normal element, exploring normal element')
    normal_element: fbx.FbxLayerElementNormal = normals
    
    if normal_element.GetMappingMode() == fbx.FbxLayerElementNormal.eByControlPoint:
        print('mapping mode is by control point')
    elif normal_element.GetMappingMode() == fbx.FbxLayerElementNormal.eByPolygon:
        print('mapping mode is by polygon')
    elif normal_element.GetMappingMode() == fbx.FbxLayerElementNormal.eByPolygonVertex:
        print('mapping mode is by polygon vertex')
    else:
        print('mapping mode of this kind is unhandled')
        
    if normal_element.GetReferenceMode() == fbx.FbxLayerElementNormal.eDirect:
        print('normal element has direct reference mode')
    elif normal_element.GetReferenceMode() == fbx.FbxLayerElementNormal.eIndex:
        print('normal element has index reference mode')
    elif normal_element.GetReferenceMode() == fbx.FbxLayerElementNormal.eIndexToDirect:
        print('normal element has index-to-direct reference mode')
    else:
        print('normal element has unhandled reference mode')
        
    print('printing normals of the cube by polygon vertex (24 in total)')
    normals_array: fbx.FbxLayerElementArrayTemplate_FbxVector4 = normal_element.GetDirectArray()
    for i in range(0, normals_array.GetCount()):
        print(normals_array.GetAt(i))

memory_manager.Destroy()

8 control points in this mesh
fbx.FbxVector4(-1.000000, 0.000000, 0.000000, 0.000000)
mesh layer 0 has normal element, exploring normal element
mapping mode is by polygon vertex
normal element has direct reference mode
printing normals of the cube by polygon vertex (24 in total)
fbx.FbxVector4(-1.000000, 0.000000, 0.000000, 0.000000)
fbx.FbxVector4(-1.000000, 0.000000, 0.000000, 0.000000)
fbx.FbxVector4(-1.000000, 0.000000, 0.000000, 0.000000)
fbx.FbxVector4(-1.000000, 0.000000, 0.000000, 0.000000)
fbx.FbxVector4(0.000000, 1.000000, 0.000000, 0.000000)
fbx.FbxVector4(0.000000, 1.000000, 0.000000, 0.000000)
fbx.FbxVector4(0.000000, 1.000000, 0.000000, 0.000000)
fbx.FbxVector4(0.000000, 1.000000, 0.000000, 0.000000)
fbx.FbxVector4(1.000000, 0.000000, 0.000000, 0.000000)
fbx.FbxVector4(1.000000, 0.000000, 0.000000, 0.000000)
fbx.FbxVector4(1.000000, 0.000000, 0.000000, 0.000000)
fbx.FbxVector4(1.000000, 0.000000, 0.000000, 0.000000)
fbx.FbxVector4(0.000000, -1.000000, 0.000000, 0.000000)
